In [1]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
prop_data = pd.read_csv('../data/all_sold_property_data.csv')

C:\Users\2003a\AppData\Local\Temp\ipykernel_12348\1899678723.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  prop_data = pd.read_csv('../data/all_sold_property_data.csv')


In [3]:
stripped_data = prop_data[['bathrooms', 'bedrooms', 'homeType', 'livingArea', 'lotSize', 'taxAssessedValue', 'yearBuilt', 'price']]
#stripped_data.set_index('address')
stripped_data = stripped_data.dropna()
stripped_data.size

56560

In [ ]:
%pip install -U scikit-learn
%pip install xgboost


In [5]:
df_encoded = pd.get_dummies(stripped_data, columns=['homeType'], drop_first=True)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

y = df_encoded['price']
df_encoded = df_encoded[['bathrooms', 'bedrooms', 'livingArea', 'lotSize', 'taxAssessedValue' , 'yearBuilt'] + 
               [col for col in df_encoded.columns if 'homeType_' in col]]
X = df_encoded[['bathrooms', 'bedrooms', 'livingArea', 'lotSize', 'yearBuilt', 'taxAssessedValue', 'homeType_MULTI_FAMILY', 'homeType_SINGLE_FAMILY']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    smape = 100 * np.mean(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return {'RMSE': rmse, 'SMAPE': smape ,'MAE': mae, 'R²': r2}

# Evaluating multiple models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

models = {
    'Linear Regres': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor()
}

# Evaluate each model and print the results
for name, model in models.items():
    metrics = evaluate_model(model, X_train, X_test, y_train, y_test)
    print(f"{name}: \tRMSE={metrics['RMSE']:.2f}, \tSAMPE={metrics['SMAPE']:.2f}%, \tMAE={metrics['MAE']:.2f}, \tR²={metrics['R²']:.2f}")


Linear Regression: 	RMSE=99414.05, 	SAMPE=19.05%, 	MAE=53385.72, 	R²=0.82
Decision Tree: 	RMSE=149253.75, 	SAMPE=24.98%, 	MAE=72189.61, 	R²=0.59
Random Forest: 	RMSE=109838.28, 	SAMPE=18.31%, 	MAE=53501.82, 	R²=0.78
XGBoost: 	RMSE=110676.39, 	SAMPE=18.58%, 	MAE=54856.06, 	R²=0.78
